# Maps around reservoirs

## To-dos
- [ ] trend maps for xarray data
- [ ] area_subset of GRACE data (instead of point subset)
- [ ] area_subset of IMERG data
- [ ] area_subset of precip data

In [ ]:
func_dir = '/global/home/users/ann_scheliga/CYGNSS-GRACE/codebase/'

In [ ]:
import sys
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt

sys.path.append(func_dir)
import load_data
import time_series_calcs
import plotting_functions
import area_subsets

In [ ]:
fw = load_data.load_CYGNSS_05()
grace_dict = load_data.load_GRACE()
res_shp = load_data.load_GRanD()
precip = load_data.load_IMERG()

## Scratchwork

In [ ]:
## Trying out plotting functionality

tws = grace_dict['cmwe'].T
x_tws= list(map(time_series_calcs.toYearFraction,tws.index))
y_tws = tws
initial_tws_slopes = time_series_calcs.linregress_wrap(x_tws,y_tws)
print(initial_tws_slopes.head())

initial_tws_slopes_gdf = gpd.GeoDataFrame(pd.concat([initial_tws_slopes , grace_dict['mascon']],axis=1))

fig , ax = plt.subplots(figsize=(12,8))
input_gdf = initial_tws_slopes_gdf
count = 0
plotting_functions.statsig_map(input_gdf,ax,count, pie_row = [0.5,0.4,0.1])

In [ ]:
## TESTING phase, idk if this works
def gpd_map_prep(input_gpd,func):
    tws = input_gpd
    x_tws= list(map(time_series_calcs.toYearFraction,tws.index))
    y_tws = tws
    initial_tws_slopes = func(x_tws,y_tws)

    # initial_tws_slopes_gdf = gpd.GeoDataFrame(pd.concat([initial_tws_slopes , input_polys],axis=1))
    # input_gdf = initial_tws_slopes_gdf
    return initial_tws_slopes
gpd_map_prep(grace_dict['cmwe'],time_series_calcs.linregress_wrap)

In [ ]:
coords_oi = reservoir_name_to_point('Nouzhadu')

test = grace_point_subset(coords_oi)
print(test[1].iloc[0]['geometry'].exterior)
test[1]['geometry'].explore()